# Pandas and SQL
Structured Query Language, or **SQL**, is the language of databases. Though "SQL" may seem new to you, you've actually used it in your GIS work, specifically in selecting records by attribute values. SQL, however, is capable of much more than just selecting records; SQL is used to create, read, relate (i.e. join), update, delete records stored in database tables. 

In this exercise we use Pandas to mimic many SQL functions, showing that we don't need to learn daunting database management systems like Oracle, Microsoft SQL Server, PostgreSQL, or even MS Access, to run some of the everyday database functions you might want to run. Instead, we can import data as a dataframe and use some Pandas functions to get what you need. 

Here we explore some useful data query and transformation techniques are applied to marine species observations data extracted from OBIS/SeaMap. These data are stored in the data folder as `Seamap400.csv`. 

→ *You may want to examine this file in Excel to familiarize yourself with the data first.*

In [25]:
#Import the modules
import pandas as pd
import numpy as np #Well need NumPy for some of its data types...

## Loading the csv data into a Pandas dataframe
We've loaded CSV data into a Pandas dataframe in previous exercises. The code block below demonstrates how the `read_csv` function easily does this. 

In [6]:
#Import the data into a dataframe called "dfSeamap"
dfSeamap = pd.read_csv('../Data/Seamap400.csv')
#Show the first 2 records in the result
dfSeamap.head(2)

,ID,datasetid,rowid,last_mod,latitude,longitude,sp_code,sp_tsn,sp_count,obs_date,scientific,sp_common,sp_class,provider
0,1,400,1114,6/15/2007 10:06:59,37.27,-75.20,BALA,180522,2,3/13/2007 14:11:00,Balaenopteridae,rorquals,Mammalia,University of North Carolina at Wilmington
1,2,400,1006,6/15/2007 10:06:59,36.95,-75.87,Bph,180527,2,2/28/2007 10:52:00,Balaenoptera physalus,Fin Whale,Mammalia,University of North Carolina at Wilmington


The result *looks* good, and indeed we can do much with this dataframe. BUT, there are some "gotchas" that we can fix:
* First, Pandas assigns its own index, when we might want to keep the ID field as the index values.
* The `dataset` and `rowid` fields, are actually nominal values, not actual numbers, and we should import these as strings, not numbers (in case they have a leading zero, for example).
* The `last_mod` and `obs_date` fields are imported as string objects, but we can import these as _date_ object, which will allow for selection via _time slices_.

In [7]:
dfSeamap.dtypes

ID              int64
datasetid       int64
rowid           int64
last_mod       object
latitude      float64
longitude     float64
sp_code        object
sp_tsn          int64
sp_count        int64
obs_date       object
scientific     object
sp_common      object
sp_class       object
provider       object
dtype: object

So... to fix this, we can specify a few extra parameters when importing the data. Below is the `read_csv()` function applied as above, but with some additional parameters (and also reformatted to make this statement more readable):
* The `index_col` specifies which column will be our dataFrame's **index**. 
* `parse_dates` indicates that the `last_mod` and `obs_date` columns are a **date fields**, not strings, which enables us to to time-based queries. 
* And finally, the `dtype` functions allows us to override the defaut data type assignments. Here, we pass a dictionary consisting of column names (keys) and data types (values) to indicate which columns:data type pairs we want to specifiy. 

In [3]:
#Import the Seamap400.csv file to a DataFrame
dfSeamap = pd.read_csv(
    '../Data/Seamap400.csv',             # File containing the data
    index_col = 'ID',                    # Which column to set to the dataframe's index
    parse_dates=['last_mod','obs_date'], # Which columns to format as dates
    dtype={'datasetid':'str',            # Coerce the 'datasetid' and 'rowid' columns to be strings
           'rowid':'str'
          }
)
#Display the first two rows
dfSeamap.head(2)

,datasetid,rowid,last_mod,latitude,longitude,sp_code,sp_tsn,sp_count,obs_date,scientific,sp_common,sp_class,provider
ID,,,,,,,,,,,,,
1,400,1114,2007-06-15 10:06:59,37.27,-75.20,BALA,180522,2,2007-03-13 14:11:00,Balaenopteridae,rorquals,Mammalia,University of North Carolina at Wilmington
2,400,1006,2007-06-15 10:06:59,36.95,-75.87,Bph,180527,2,2007-02-28 10:52:00,Balaenoptera physalus,Fin Whale,Mammalia,University of North Carolina at Wilmington


Other than the ID now being used as the index the result looks the same, but if you inspect the data types, you'll see we got what we wanted.

In [4]:
#Check the data types of the import
dfSeamap.dtypes

datasetid             object
rowid                 object
last_mod      datetime64[ns]
latitude             float64
longitude            float64
sp_code               object
sp_tsn                 int64
sp_count               int64
obs_date      datetime64[ns]
scientific            object
sp_common             object
sp_class              object
provider              object
dtype: object

## Exploring our dataFrame
Now, let's examine how many unique species there are. This is analogous to the `SELECT DISTINCT` SQL operation.

In [8]:
dfSeamap['sp_common'].unique()

array(['rorquals', 'Fin Whale', 'Loggerhead', 'cartilaginous fishes',
       'Basking Shark', 'Common Dolphin', 'dolphins',
       'Northern Right Whale', 'Short-finned Pilot Whale',
       'Ocean Sunfish', 'Atlantic Ridley', 'Atlantic Manta',
       'Humpback Whale', 'cow-nose rays', 'Bottlenose Dolphin',
       'Sea Turtles'], dtype=object)

## Selecting <u>columns</u> of data from a dataFrame
Selecting columns is analogous to the SQL `SELECT ... FROM` clause, but the syntax in Pandas is a bit different.

#### ♦ Selecting data from <u>a single column</u> in the dataframe
Selecting a single column is easy as specifying the column name, either between brackets with the field name in quotes, or using the dot notation. (*The former is preferred, in case your field name is the the same as another property or method of a dataFrame.*)

When only one column is selected, the object returned is actually not a dataFrame, but a pandas **series** object, which is quite similar to a one-dimensional NumPy array. (However, in Pandas, series are limited to one dimension...)

In [9]:
#Select just the sp_common field into a new view
dfSelect = dfSeamap['sp_common']
dfSelect.head()

0     rorquals
1    Fin Whale
2    Fin Whale
3    Fin Whale
4    Fin Whale
Name: sp_common, dtype: object

In [10]:
#Using "dot notation" (<dataframe>.<field name>) also works
commonNames = dfSeamap.sp_common
type(commonNames)

pandas.core.series.Series

In [11]:
#As the object returned is a series, we can get records from its index
commonNames[:8] #Returns the first 8 records

0     rorquals
1    Fin Whale
2    Fin Whale
3    Fin Whale
4    Fin Whale
5    Fin Whale
6    Fin Whale
7    Fin Whale
Name: sp_common, dtype: object

In [13]:
#What code would you use to get the last 8 records? (Replace the █)
commonNames[-8:]

2261    Sea Turtles
2262    Sea Turtles
2263    Sea Turtles
2264    Sea Turtles
2265    Sea Turtles
2266    Sea Turtles
2267    Sea Turtles
2268    Sea Turtles
Name: sp_common, dtype: object

#### ♦ Selecting data  from <u>multiple</u> columns
Selecting multiple columns can be done by including a list of the column names. *Note the double set of brackets: the outer set is the syntax of the dataFrame object, and the inner set denotes the list of field names we want.*

Here, the object returned resembles a dataFrame, not a series (since Pandas' series can only be one-dimension). However, in actuality, this is simply a **view** of the original dataFrame, not a new object - much like query views in SQL...

In [14]:
#Select the 'sp_common' and 'sp_class' columns into a new view
dfSelect = dfSeamap[['sp_common','sp_class']]
dfSelect.head()

,sp_common,sp_class
0,rorquals,Mammalia
1,Fin Whale,Mammalia
2,Fin Whale,Mammalia
3,Fin Whale,Mammalia
4,Fin Whale,Mammalia


In [15]:
##What code would create a table of just the species common name and the latitude and longitude columns?
dfSelect2 = dfSeamap[['sp_common', 'latitude','longitude']]
dfSelect2.head()

,sp_common,latitude,longitude
0,rorquals,37.27,-75.20
1,Fin Whale,36.95,-75.87
2,Fin Whale,36.96,-75.43
3,Fin Whale,36.95,-75.49
4,Fin Whale,36.70,-75.55


## Selecting <u>rows</u> in a dataFrame
Selecting rows, analogous to the `select...where` statement in SQL, can be done by creating **boolean masks** of the criteria you want and then applying those masks. This can be done explicitily as two steps, or more commonly in a single, compount step. 

Before making our selection, let's examine how we can easily extract a list of valid options using the `unique` property.

In [16]:
#List the unique values in the 'sp_class' field.
dfSeamap['sp_class'].unique()

array(['Mammalia', 'Reptilia', 'Chondrichthyes', 'Actinopterygii'],
      dtype=object)

In [17]:
#Display just the *number* of unique latitude values
dfSeamap['latitude'].nunique()

267

#### Selecting rows in two steps:

In [18]:
#Step 1: Create the boolean mask
theMask = dfSeamap['sp_class'] == 'Mammalia'
theMask[2150:2156] #Show a selection of records, to display values

2150     True
2151     True
2152     True
2153    False
2154    False
2155    False
Name: sp_class, dtype: bool

In [19]:
#Step 2: Applying the mask to return only 'true' records
dfMammals = dfSeamap[theMask]
dfMammals.sample(4) #Show a random sample from the product

,ID,datasetid,rowid,last_mod,latitude,longitude,sp_code,sp_tsn,sp_count,obs_date,scientific,sp_common,sp_class,provider
2145,2146,400,2251,6/15/2007 10:06:59,36.52,-75.80,Ttr,180426,2,5/1/2007 10:38:00,Tursiops truncatus,Bottlenose Dolphin,Mammalia,University of North Carolina at Wilmington
1170,1171,400,793,6/15/2007 10:06:59,34.57,-76.48,DELP,180415,4,2/17/2007 10:50:00,Delphinidae,dolphins,Mammalia,University of North Carolina at Wilmington
2143,2144,400,2239,6/15/2007 10:06:59,36.20,-75.73,Ttr,180426,10,4/30/2007 13:59:00,Tursiops truncatus,Bottlenose Dolphin,Mammalia,University of North Carolina at Wilmington
1517,1518,400,508,6/15/2007 10:06:59,34.47,-77.43,Ttr,180426,6,2/4/2007 12:34:00,Tursiops truncatus,Bottlenose Dolphin,Mammalia,University of North Carolina at Wilmington


In [20]:
#Proof that we only got mammals in the result
dfMammals['sp_class'].unique()

array(['Mammalia'], dtype=object)

#### Selecting rows in one step:

In [21]:
#Get all the mammals and report all unique scientific names
dfMammals = dfSeamap[dfSeamap['sp_class'] == 'Mammalia']
dfMammals.shape #Reveal the size of the result

(929, 14)

In [22]:
dfMammals['scientific'].unique()

array(['Balaenopteridae', 'Balaenoptera physalus', 'Delphinus delphis',
       'Delphinidae', 'Eubalaena glacialis', 'Globicephala macrorhynchus',
       'Megaptera novaeangliae', 'Tursiops truncatus'], dtype=object)

#### Selecting rows using `query`:
We can also execute familiar **query strings** to extract records. *Personally, I don't like this method as the query strings tend to be finicky and can be tough to use with quotes..*

In [23]:
dfMammals = dfSeamap.query("sp_class == 'Mammalia'")
dfMammals.shape

(929, 14)

#### Complex row selections
We can combine masks using logical operators (`&` = "and", `|` = "or").

In [26]:
#Select mammal records in the 2nd half of 2006
mammalMask = dfSeamap['sp_class'] == 'Mammalia'

#Create date objects for the start and end dates
startDate = np.datetime64('2006-07-01')
endDate = np.datetime64('2007-01-01')

#Create the date masks
startMask = (dfSeamap['obs_date'] >= startDate) 
endMask = (dfSeamap['obs_date'] < endDate)

#Apply the masks, using the bitwise '&' to return rows where all masks are true
dfSelect2 = dfSeamap[mammalMask & startMask & endMask]
dfSelect2['scientific'].unique()

TypeError: '<=' not supported between instances of 'numpy.ndarray' and 'str'

#### Return rows matching a substring
The `.str` function on a column allows us to use some string operations  on the values in that field.  Here we use the string `startswith` function to return all rows where the row's value starts with 'Delphin'. See https://pandas.pydata.org/pandas-docs/stable/text.html for other string operations.

In [27]:
#Select rows where the scientific name starts with "Delphin" 
dolphinMask = dfSeamap['scientific'].str.startswith('Delphin')
dfDolphins = dfSeamap[dolphinMask]

#Use the nunique function to just return the number 
#  of unique scientific names
dfDolphins['scientific'].nunique()

2

#### More complex queries with `apply` and  `lambda`
For maximum flexibility, we can actually write our own functions to be applied to each value in a column (or multiple columns). This is done using the `apply` function to a dataFrame and then specifying the subcode we want to use with Python's `lambda` statement. (This seems fairly complex at first, but it actually somewhat straightforward -- and can be very useful...)

In [28]:
#Create a mask by searching each row for the string "Whale" and apply the mask
# to list the scientific names of these records
whaleMask = dfSeamap['sp_common'].apply(lambda x: 'Whale' in x)
dfWhale = dfSeamap[whaleMask]
dfWhale['scientific'].unique()

array(['Balaenoptera physalus', 'Eubalaena glacialis',
       'Globicephala macrorhynchus', 'Megaptera novaeangliae'],
      dtype=object)

## Grouping/Aggregating data
Pandas can aggregate data on values like SQL as well. We do this with the `groupby` statement

In [29]:
#Count the observations by common name
grpSpCommon = dfSeamap.groupby('sp_common')
grpSpCommon['sp_common'].count()

sp_common
Atlantic Manta               24
Atlantic Ridley               5
Basking Shark                29
Bottlenose Dolphin          849
Common Dolphin               16
Fin Whale                     8
Humpback Whale                4
Loggerhead                  957
Northern Right Whale         11
Ocean Sunfish                64
Sea Turtles                 116
Short-finned Pilot Whale      2
cartilaginous fishes        143
cow-nose rays                 2
dolphins                     38
rorquals                      1
Name: sp_common, dtype: int64

We can also show *all* the summary stats with the dataFrame's `describe` function.

In [30]:
grpSpCommon.describe()

ID                                            \
                          count         mean         std     min      25%   
sp_common                                                                   
Atlantic Manta             24.0  1223.500000    7.071068  1212.0  1217.75   
Atlantic Ridley             5.0  1209.000000    1.581139  1207.0  1208.00   
Basking Shark              29.0  1124.000000    8.514693  1110.0  1117.00   
Bottlenose Dolphin        849.0  1729.000000  245.229484  1305.0  1517.00   
Common Dolphin             16.0  1146.500000    4.760952  1139.0  1142.75   
Fin Whale                   8.0     5.500000    2.449490     2.0     3.75   
Humpback Whale              4.0  1300.500000    1.290994  1299.0  1299.75   
Loggerhead                957.0   488.000000  276.406404    10.0   249.00   
Northern Right Whale       11.0  1198.000000    3.316625  1193.0  1195.50   
Ocean Sunfish              64.0  1266.046875   19.718197  1206.0  1250.75   
Sea Turtles               116.0  2211.500000   33.630343  2154.0  2182.75   
Short-finned Pilot Whale    2.0  1204.500000    0.707107  1204.0  1204.25   
cartilaginous fishes      143.0  1038.000000   41.424630   967.0  1002.50   
cow-nose rays               2.0  1303.500000    0.707107  1303.0  1303.25   
dolphins                   38.0  1173.500000   11.113055  1155.0  1164.25   
rorquals                    1.0     1.000000         NaN     1.0     1.00   

                                                  datasetid         ...  \
                             50%      75%     max     count   mean  ...   
sp_common                                                           ...   
Atlantic Manta            1223.5  1229.25  1235.0      24.0  400.0  ...   
Atlantic Ridley           1209.0  1210.00  1211.0       5.0  400.0  ...   
Basking Shark             1124.0  1131.00  1138.0      29.0  400.0  ...   
Bottlenose Dolphin        1729.0  1941.00  2153.0     849.0  400.0  ...   
Common Dolphin            1146.5  1150.25  1154.0      16.0  400.0  ...   
Fin Whale                    5.5     7.25     9.0       8.0  400.0  ...   
Humpback Whale            1300.5  1301.25  1302.0       4.0  400.0  ...   
Loggerhead                 488.0   727.00   966.0     957.0  400.0  ...   
Northern Right Whale      1198.0  1200.50  1203.0      11.0  400.0  ...   
Ocean Sunfish             1266.5  1282.25  1298.0      64.0  400.0  ...   
Sea Turtles               2211.5  2240.25  2269.0     116.0  400.0  ...   
Short-finned Pilot Whale  1204.5  1204.75  1205.0       2.0  400.0  ...   
cartilaginous fishes      1038.0  1073.50  1109.0     143.0  400.0  ...   
cow-nose rays             1303.5  1303.75  1304.0       2.0  400.0  ...   
dolphins                  1173.5  1182.75  1192.0      38.0  400.0  ...   
rorquals                     1.0     1.00     1.0       1.0  400.0  ...   

                            sp_tsn           sp_count                        \
                               75%       max    count       mean        std   
sp_common                                                                     
Atlantic Manta            160992.0  160992.0     24.0   1.291667   1.082636   
Atlantic Ridley           551770.0  551770.0      5.0   1.200000   0.447214   
Basking Shark             159907.0  159907.0     29.0   1.655172   1.369981   
Bottlenose Dolphin        180426.0  180426.0    849.0   6.914016  10.393934   
Common Dolphin            180438.0  180438.0     16.0  17.312500  19.296697   
Fin Whale                 180527.0  180527.0      8.0   3.250000   1.982062   
Humpback Whale            180530.0  180530.0      4.0   1.000000   0.000000   
Loggerhead                173830.0  173830.0    957.0   1.334378   1.206242   
Northern Right Whale      180537.0  180537.0     11.0   2.272727   1.489356   
Ocean Sunfish             173414.0  173414.0     64.0   1.109375   0.403051   
Sea Turtles               173828.0  173828.0    116.0   1.094828   0.348374   
Short-finned Pilot Whale  180466.

## Reshaping tables
In addition to grouping data, we can "pivot" the data, summarizing by one field and aggregating values across other fields. Let's look at an example where we display 

In [31]:
#Make a copy of the data
dfSeamapCopy = dfSeamap.copy(deep=True)

In [32]:
#Add a "day" column (https://pandas.pydata.org/pandas-docs/stable/api.html#datetimelike-properties)
dfSeamapCopy['week_num'] = dfSeamapCopy['obs_date'].dt.weekofyear
dfSeamapCopy.head()

AttributeError: Can only use .dt accessor with datetimelike values

In [33]:
#Pivot to a table listing row=species, col = day of year, values = latitude
dfLat = dfSeamapCopy.pivot_table(index='sp_common',columns='week_num',values='latitude')
dfLat.head()

KeyError: 'week_num'

In [ ]:
#Plot the longitudinal travel across weeks of a given species
dfLat.loc['Bottlenose Dolphin'].plot(kind='line')

## Joining tables
Also like SQL, pandas can join tables. Below we'll create two tables from our aggregated data: one will list the minimum of the latitude and longitude columns, and the second will list the maximum values. Then we'll join these two tables and compute the geographic extent of each species observations. 

In [34]:
#Create dataFrames of the minimum and then maximum of the lat and lng fields
minCoords = grpSpCommon['latitude','longitude'].min()
maxCoords = grpSpCommon['latitude','longitude'].max()

In [35]:
# Have a look at what is returned (for the min table)
minCoords.head()

,latitude,longitude
sp_common,,
Atlantic Manta,34.20,-76.48
Atlantic Ridley,33.52,-77.53
Basking Shark,34.14,-77.31
Bottlenose Dolphin,33.41,-78.40
Common Dolphin,33.45,-77.47


In [36]:
#Use the Pandas 'merge' command to join the two tables
sppExtent = pd.merge(left=minCoords,     #Specifies the left table
                     right=maxCoords,     #Specifies the right table
                     how = 'inner',      #Specifies the type of join
                     left_index=True,    #Use the index of the left table as the join item
                     right_index=True)   #Use the index of the right table as the join item
#Have a look
sppExtent.head()

,latitude_x,longitude_x,latitude_y,longitude_y
sp_common,,,,
Atlantic Manta,34.20,-76.48,35.60,-74.86
Atlantic Ridley,33.52,-77.53,35.67,-74.95
Basking Shark,34.14,-77.31,35.67,-75.19
Bottlenose Dolphin,33.41,-78.40,37.40,-74.89
Common Dolphin,33.45,-77.47,35.67,-74.92


In [37]:
#Rename the columns to the values in the list provided
sppExtent.columns = ['minX','minY','maxX','maxY']
sppExtent.head()

,minX,minY,maxX,maxY
sp_common,,,,
Atlantic Manta,34.20,-76.48,35.60,-74.86
Atlantic Ridley,33.52,-77.53,35.67,-74.95
Basking Shark,34.14,-77.31,35.67,-75.19
Bottlenose Dolphin,33.41,-78.40,37.40,-74.89
Common Dolphin,33.45,-77.47,35.67,-74.92


In [38]:
#Compute two new columns as the difference between max and min
sppExtent['XRange'] = sppExtent.maxX - sppExtent.minX
sppExtent['YRange'] = sppExtent.maxY - sppExtent.minY
sppExtent.head()

,minX,minY,maxX,maxY,XRange,YRange
sp_common,,,,,,
Atlantic Manta,34.20,-76.48,35.60,-74.86,1.40,1.62
Atlantic Ridley,33.52,-77.53,35.67,-74.95,2.15,2.58
Basking Shark,34.14,-77.31,35.67,-75.19,1.53,2.12
Bottlenose Dolphin,33.41,-78.40,37.40,-74.89,3.99,3.51
Common Dolphin,33.45,-77.47,35.67,-74.92,2.22,2.55
